In [1]:
############ IMPORTS ####################
import sys
sys.path.append("./")
import os
from os import path
import numpy as np
import random
import torch
import torch.utils.data as dataf
import torch.nn as nn
import matplotlib.pyplot as plt
from scipy import io as sio
from sklearn.decomposition import PCA
from torch.nn.parameter import Parameter
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import time
from PIL import Image
import math
from sklearn.model_selection import train_test_split
from operator import truediv
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
from torchsummary import summary
import record

In [2]:
def loadData(name):
    
    if name == "Houston":
        dataHSI = sio.loadmat('./../Houston/houston.mat')['houston']

        dataLIDAR = np.array(Image.open('./../Houston/houston_lidar.tif'))
        dataLIDAR = dataLIDAR.reshape(dataLIDAR.shape[0],dataLIDAR.shape[1],1)

        
        labels = sio.loadmat('./../Houston/houston_gt.mat')['houston_gt_te']
        
        labels += sio.loadmat('./../Houston/houston_gt.mat')['houston_gt_tr']

        
    elif(name == "Trento"):
        
        dataHSI = sio.loadmat('./../Trento/HSI.mat')['HSI']

        dataLIDAR = sio.loadmat('./../Trento/LiDAR.mat')['LiDAR']
        dataLIDAR = dataLIDAR.reshape(dataLIDAR.shape[0],dataLIDAR.shape[1],1)
        
        labels = sio.loadmat('./../Trento/TSLabel.mat')['TSLabel']
        labels += sio.loadmat('./../Trento/TRLabel.mat')['TRLabel']
        
    elif(name == "MUUFL"):
        
        dataHSI = sio.loadmat('./../MUUFL/muufl_share.mat')['hsi_img']

        dataLIDAR = sio.loadmat('./../MUUFL/muufl_share.mat')['lidarz']
        labels = sio.loadmat('./../MUUFL/muufl_share.mat')['labels']
    return dataHSI,dataLIDAR, labels


def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

def normalizeHSI(X):
    for i in range(X.shape[2]):
        minimal = X[:, :, i].min()
        maximal = X[:, :, i].max()
        X[:, :, i] = (X[:, :, i] - minimal)/(maximal - minimal)
    return X
        
def normalizeLIDAR(X):
    minimal = X.min()
    maximal = X.max()
    X = (X - minimal)/(maximal - minimal)
    return X

def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    print(zeroPaddedX.shape)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]), dtype=int)
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        
    return patchesData, patchesLabels


In [3]:
class Conv3d_cd(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1,
                 padding=1, dilation=1, groups=1, bias=False, theta=0.7):

        super(Conv3d_cd, self).__init__() 
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.theta = theta

    def forward(self, x):
        out_normal = self.conv(x)


        if math.fabs(self.theta - 0.0) < 1e-8:
            return out_normal 
        else:
            #pdb.set_trace()
            [C_out,C_in, kernel_size,kernel_size,depth] = self.conv.weight.shape
            kernel_diff = self.conv.weight.sum(2).sum(2)
            kernel_diff = kernel_diff[:, :, None, None]
            kernel_diff = kernel_diff.repeat(1,1,3,3,1)
            out_diff = F.conv3d(input=x, weight=kernel_diff, bias=self.conv.bias, stride=self.conv.stride, padding=self.conv.padding, groups=self.conv.groups)

            return out_normal - self.theta * out_diff

In [8]:
class CNN(nn.Module):
    def __init__(self, FM, Classes, patchsize, NC):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            Conv3d_cd(
                in_channels = 1,
                out_channels = FM,
                kernel_size = (3, 3, 7),
                stride = 1,
                padding = (0,0,0)
            ),
            nn.BatchNorm3d(FM),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1,1,2)),
#             nn.Dropout(0.5),
        )
        self.final_bands = (NC - 6) // 2
        
        self.conv2 = nn.Sequential(
            Conv3d_cd(FM, FM*2, (3, 3, 7 ), 1, (0,0,0)),
            nn.BatchNorm3d(FM*2),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1,1,2))

        )
        self.final_bands = (self.final_bands - 6) // 2
        self.conv3 = nn.Sequential(
            Conv3d_cd(FM*2, FM*4, (3, 3, 7), 1, (0,0,0)),
            nn.BatchNorm3d(FM*4),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1,1,2))

        )
        self.final_bands = (self.final_bands - 6) // 2
        
        self.final_patch_size = patchsize - 6
        
        print(self.final_bands)
        print(self.final_patch_size)
        self.out1 =  nn.Linear(self.final_patch_size * self.final_patch_size * FM * 4 * self.final_bands, Classes)
        
    def forward(self, x1):
        x1 = x1.unsqueeze(1)
        x1 = self.conv1(x1)
        x1 = self.conv2(x1)
        x1 = self.conv3(x1)
        x1 = x1.reshape(x1.shape[0], -1)  # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        out1 = self.out1(x1)
        return out1

In [15]:
# CONFIGS
os.environ["CUDA_VISIBLE_DEVICES"]="1"
datasetNames = ["Houston"]
testSizeNumber = 2500
batchsize = 64
EPOCH = 200
LR = 0.001
trainRatios = [0.1]
patchSizes = [11]
NUM_ITERATIONS = 3
FM = 16


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

def reports (xtest,ytest,name):
    pred_y = np.empty((len(ytest)), dtype=np.float32)
    number = len(ytest) // testSizeNumber
    for i in range(number):
        temp = xtest[i * testSizeNumber:(i + 1) * testSizeNumber, :, :, :]
        temp = temp.cuda()

        temp2 = cnn(temp)
        temp3 = torch.max(temp2, 1)[1].squeeze()
        pred_y[i * testSizeNumber:(i + 1) * testSizeNumber] = temp3.cpu()
        del temp, temp2, temp3

    if (i + 1) * testSizeNumber < len(ytest):
        temp = xtest[(i + 1) * testSizeNumber:len(ytest), :, :, :]
        temp = temp.cuda()


        temp2 = cnn(temp)
        temp3 = torch.max(temp2, 1)[1].squeeze()
        pred_y[(i + 1) * testSizeNumber:len(ytest)] = temp3.cpu()
        del temp, temp2, temp3

    pred_y = torch.from_numpy(pred_y).long()
    
    if name == 'Houston':
        target_names = ['Healthy grass', 'Stressed grass', 'Synthetic grass'
                        ,'Trees', 'Soil', 'Water', 
                        'Residential', 'Commercial', 'Road', 'Highway',
                        'Railway', 'Parking Lot 1', 'Parking Lot 2', 'Tennis Court',
                        'Running Track']
    elif name == 'Trento':
        target_names = ['Apples','Buildings','Ground','Woods','Vineyard',
                        'Roads']
    elif name == 'MUUFL':
        target_names = ['Trees','Grass_Pure','Grass_Groundsurface','Dirt_And_Sand', 'Road_Materials','Water',"Buildings'_Shadow",
                    'Buildings','Sidewalk','Yellow_Curb','ClothPanels']
    elif name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed', 
                'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                'Stone-Steel-Towers']
    elif name == 'SA':
        target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
    elif name == 'UP':
        target_names = ['Asphalt','Meadows','Gravel','Trees', 'Painted metal sheets','Bare Soil','Bitumen',
                        'Self-Blocking Bricks','Shadows']

    
    oa = accuracy_score(ytest, pred_y)
    confusion = confusion_matrix(ytest, pred_y)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(ytest, pred_y)

    return confusion, oa*100, each_acc*100, aa*100, kappa*100




for patchsize in patchSizes:
    for trainRatio in trainRatios:
        for datasetName in datasetNames:
            print("Current patch size = ",patchsize)
            print("Current training ratio = ",trainRatio)
            print("Current dataset = ",datasetName)
            
            try:
                os.makedirs(datasetName)
            except FileExistsError:
                pass
            
            X1, _, y = loadData(datasetName)
            X1 = normalizeHSI(X1)

            X1, yPatch = createImageCubes(X1, y, windowSize=patchsize)
            TrainPatch, TestPatch, TrainLabel, TestLabel = splitTrainTestSet(X1, yPatch, 1. - trainRatio, randomState= 42)
            TrainPatch = TrainPatch.astype(np.float32)
            TestPatch = TestPatch.astype(np.float32)
            NC = TrainPatch.shape[3]

            TrainPatch = torch.from_numpy(TrainPatch)
            TrainLabel = torch.from_numpy(TrainLabel)-1
            TrainLabel = TrainLabel.long()
            TestPatch = torch.from_numpy(TestPatch)
            TestLabel = torch.from_numpy(TestLabel)-1
            TestLabel = TestLabel.long()

            Classes = len(np.unique(TrainLabel))

            dataset = dataf.TensorDataset(TrainPatch, TrainLabel)
            train_loader = dataf.DataLoader(dataset, batch_size=batchsize, shuffle=True)

            print("Train data shape = ", TrainPatch.shape)
            print("Train label shape = ", TrainLabel.shape)
            print("Test data shape = ", TestPatch.shape)
            print("Test label shape = ", TestLabel.shape)

            KAPPA = []
            OA = []
            AA = []
            ELEMENT_ACC = np.zeros((NUM_ITERATIONS, Classes))

            for iterNum in range(NUM_ITERATIONS):    
                cnn = CNN(FM, Classes, patchsize,NC)
                cnn = cnn.cuda()
                summary(cnn, (patchsize, patchsize, NC))
                optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
                loss_func = nn.CrossEntropyLoss()  # the target label is not one-hotted

                BestAcc = 0
                torch.cuda.synchronize()
                start = time.time()

                # train and test the designed model
                for epoch in range(EPOCH):
                    for step, (b_x1,b_y) in enumerate(train_loader):
                        # move train data to GPU
                        b_x1 = b_x1.cuda()
                        b_y = b_y.cuda()

                        out1 = cnn(b_x1)
                        loss = loss_func(out1, b_y)

                        optimizer.zero_grad()  # clear gradients for this training step
                        loss.backward()  # backpropagation, compute gradients
                        optimizer.step()  # apply gradients

                        if step == len(train_loader) - 1:
                            cnn.eval()
                            pred_y = np.empty((len(TestLabel)), dtype='float32')
                            number = len(TestLabel) // testSizeNumber
                            for i in range(number):
                                temp = TestPatch[i * testSizeNumber:(i + 1) * testSizeNumber, :, :, :]
                                temp = temp.cuda()

                                temp2 = cnn(temp)
                                temp3 = torch.max(temp2, 1)[1].squeeze()
                                pred_y[i * testSizeNumber:(i + 1) * testSizeNumber] = temp3.cpu()
                                del temp, temp2, temp3


                            if (i + 1) * testSizeNumber < len(TestLabel):
                                temp = TestPatch[(i + 1) * testSizeNumber:len(TestLabel), :, :, :]
                                temp = temp.cuda()

                                temp2 = cnn(temp)
                                temp3 = torch.max(temp2, 1)[1].squeeze()
                                pred_y[(i + 1) * testSizeNumber:len(TestLabel)] = temp3.cpu()
                                del temp, temp2, temp3

                            pred_y = torch.from_numpy(pred_y).long()
                            accuracy = torch.sum(pred_y == TestLabel).type(torch.FloatTensor) / TestLabel.size(0)

                            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.cpu().numpy(), '| test accuracy: %.2f' % accuracy)

                            # save the parameters in network
                            if accuracy > BestAcc:
                                BestAcc = accuracy
                                torch.save(cnn.state_dict(), datasetName+'/net_params_checkpoint.pkl')
                            cnn.train()

                torch.cuda.synchronize()
                end = time.time()
                print("Time taken to train = ",end - start, "s")
                Train_time = end - start

                cnn.load_state_dict(torch.load(datasetName+'/net_params_checkpoint.pkl'))
                cnn.eval()


                confusion, oa, each_acc, aa, kappa = reports(TestPatch,TestLabel,datasetName)
                KAPPA.append(kappa)
                OA.append(oa)
                AA.append(aa)
                ELEMENT_ACC[iterNum, :] = each_acc
                torch.save(cnn, datasetName+'/best_model_G2C-Conv3D-HSI_Iter'+str(iterNum)+'.pt')
            print("--------" + datasetName + " Training Finished-----------")
            record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/G2C-Conv3D-HSI_Report_' + datasetName +'.txt')
                        
                        
        

Current patch size =  11
Current training ratio =  0.1
Current dataset =  Houston
(359, 1915, 144)
Train data shape =  torch.Size([1502, 11, 11, 144])
Train label shape =  torch.Size([1502])
Test data shape =  torch.Size([13527, 11, 11, 144])
Test label shape =  torch.Size([13527])
------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 9, 9, 69]        --
|    └─Conv3d_cd: 2-1                    [-1, 16, 9, 9, 138]       --
|    |    └─Conv3d: 3-1                  [-1, 16, 9, 9, 138]       1,008
|    └─BatchNorm3d: 2-2                  [-1, 16, 9, 9, 138]       32
|    └─ReLU: 2-3                         [-1, 16, 9, 9, 138]       --
|    └─MaxPool3d: 2-4                    [-1, 16, 9, 9, 69]        --
├─Sequential: 1-2                        [-1, 32, 7, 7, 31]        --
|    └─Conv3d_cd: 2-5                    [-1, 32, 7, 7, 63

/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Conv3d_cd. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
